In [16]:
#importing libraries
import pandas as pd
import re
import time  

In [17]:
#time for calculating the completing execution of the programme
start_time = time.clock()

In [18]:
#reading the data file
df = pd.read_csv("D:\Projects\Final_Year_project\Twiiter_sentiment_analysis\Data_Sourcing\Twitter_data_sabrimalai_issue.csv",encoding='UTF-8')
df.head()

,Unnamed: 0,Date,Text,Favourites,Retweet,Id,User
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,11/02/2019 13:03,b'@republic I will never stand and support thi...,0.0,0.0,1.09495e+18,iVimalNair
2,2,NaN,NaN,NaN,NaN,NaN,NaN
3,3,11/02/2019 06:48,b'@surnell @neha_aks @Sangachhwadham @Kuvalaya...,2.0,1.0,1.09485e+18,ShashiGangadhar
4,4,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Converting the text column into the string object
df["Text"] = df["Text"].astype(str)

#Deleting the null value row wise
df.dropna(inplace=True)

#deleting the Unamed column which is of no use from the file a
df.drop(["Unnamed: 0","Id"],inplace=True,axis=1)

#Rearranging the index values
df.reset_index(drop=True,inplace=True)

#converting the date_column into date object
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%d/%b/%Y")
df.head()

,Date,Text,Favourites,Retweet,User
0,02/Nov/2019,b'@republic I will never stand and support thi...,0.0,0.0,iVimalNair
1,02/Nov/2019,b'@surnell @neha_aks @Sangachhwadham @Kuvalaya...,2.0,1.0,ShashiGangadhar
2,02/Nov/2019,b'@Kuvalayamala @shenoy70 @narendramodi If ne...,0.0,0.0,Patriotofbharat
3,02/Nov/2019,b'Why there is no news about Sabarimala Karma ...,0.0,0.0,ShashiGangadhar
4,02/Oct/2019,b'@RiaRevealed @ShashiGangadhar @sambitswaraj ...,0.0,0.0,ur_vishnu


In [20]:
def data_clean(x):
    #for removing twitter handles, url's, Punctuations, Numbers, and Special Characters
    stage_1 = re.sub("(@[A-Za-z0-9_]+|https?://[A-Za-z0-9./]+|[^a-zA-Z])",' ',x)
    
    #for removing hex characters/UTF-8 BOM (Byte Order Mark)
    stage_2 = re.sub(r"(^b\s|\sn\sn|xe|xa|\s{3,5})",' ',stage_1)
    stage_3 = re.sub(r"(^\s|x$|Sabarimala|SabkaSabarimalaTemple)",' ',stage_2)
    
    #for removing short words/stop words with a length of less than 3 
    stage_4 = ' '.join([w for w in stage_3.split() if len(w) >= 3])
    
    return stage_4

df["Text"] = df["Text"].apply(lambda x:data_clean(x))

In [21]:
df.head()

,Date,Text,Favourites,Retweet,User
0,02/Nov/2019,will never stand and support this crook Always...,0.0,0.0,iVimalNair
1,02/Nov/2019,For,2.0,1.0,ShashiGangadhar
2,02/Nov/2019,neo judiciary and neo rulers let down SADHU an...,0.0,0.0,Patriotofbharat
3,02/Nov/2019,Why there news about Karma Samithi young women...,0.0,0.0,ShashiGangadhar
4,02/Oct/2019,Please understand that there gender discrim,0.0,0.0,ur_vishnu


In [22]:
#removing the rows whose text length is of 3
df = df[df['Text'].apply(lambda x: len(x.split())>1)]

#re arranging the index row
df.reset_index(drop=True,inplace=True)

In [23]:
#for adding the sentiment value and class column of each text 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [24]:
#function for getting sentiment value
def sentiment(text):
    polarity = analyzer.polarity_scores(text)
    
    if polarity['compound'] >= 0.05:
        return 1
    elif polarity['compound'] <= -0.05:
        return -1
    else :
        return 0

#function for getting sentiment class
def sentiment_class(x):
    if (x == 1):
        return "Positive"
    elif (x == -1):
        return "Negative"
    else:
        return "Neutral"

In [25]:
#Applying functions to the data for creating sentiment value and class column
df['Sentiment'] = df["Text"].apply(lambda x:sentiment(x))
df["Sentiment Class"] = df["Sentiment"].apply(lambda x: sentiment_class(x))
df["Sentiment"].value_counts()

 1    360
 0    127
-1     83
Name: Sentiment, dtype: int64

In [26]:
#splitting the text into words for stemming/lemmatization
tokenized_tweet = df['Text'].apply(lambda x: x.split())
tokenized_tweet.head()

#stemming process
#from nltk.stem.porter import *
#stemmer = PorterStemmer()
#tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
#tokenized_tweet.head()

#Lemmatization process
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i) for i in x]) 
tokenized_tweet.head()

#stitching these tokens back together and forming a text
for i in range(0,len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
df['Text'] = tokenized_tweet
df.head()

,Date,Text,Favourites,Retweet,User,Sentiment,Sentiment Class
0,02/Nov/2019,will never stand and support this crook Always...,0.0,0.0,iVimalNair,-1,Negative
1,02/Nov/2019,neo judiciary and neo ruler let down SADHU and...,0.0,0.0,Patriotofbharat,0,Neutral
2,02/Nov/2019,Why there news about Karma Samithi young woman...,0.0,0.0,ShashiGangadhar,0,Neutral
3,02/Oct/2019,Please understand that there gender discrim,0.0,0.0,ur_vishnu,1,Positive
4,02/Oct/2019,Why people comment issue which they have idea ...,0.0,0.0,Dileeplive,0,Neutral


In [27]:
df.to_csv("cleaned_sabrimalai_data.csv",encoding='utf-8',index=False)

In [28]:
print("data Cleaning is done")
end_time = time.clock()-start_time
print("Total time taken for cleaning the data {} seconds".format(end_time))

data Cleaning is done
Total time taken for cleaning the data 0.5624414700107536 seconds
